In [ ]:
//default_exp util

# Util

> Low level functions needed to build a neural net.

In [1]:
import {testEq} from './src/testutil.module.js'

In [2]:
/**
Round `x` (or all elements of `x`) to `dp` decimal places.
*/
function round(x,dp) {
    dp = dp || 0;
    if (Array.isArray(x)) {
        return x.map(_x => round(_x,dp));
    }
    return Math.round(x*Math.pow(10,dp))/Math.pow(10,dp);
}

In [3]:
testEq(1,round(1.2345));
testEq(1,round(1.2345,0));
testEq(1.2,round(1.2345,1));
testEq(1.23,round(1.2345,2));
testEq(1.235,round(1.2345,3));
testEq(1.2345,round(1.2345,4));
testEq(1.2345,round(1.2345,5));

testEq([1.2],round([1.2345],1));
testEq([1.2, 2.2],round([1.2345, 2.19],1));

In [4]:
/**
Logit (AKA log-odds) is the logarithm of the odds where p is a probability.
*/
function logit(p) {
    if (Array.isArray(p)) {
        return p.map(a=>logit(a));
    }
    return Math.log(p/(1-p));
}

In [5]:
testEq(
    [NaN, -Infinity, -1.0986, 0, 1.0986, Infinity, NaN],
    round(logit([-0.25, 0.00, 0.25, 0.50, 0.75, 1.00, 1.25]),4));

In [6]:
/**
Flatten a 2d array into a 1d array.
*/
function flatten(a2d) {
    return [].concat(...a2d);
}

In [7]:
/**
*/
function exp(a) {
    return Math.pow(Math.E, a);
}

In [8]:
/**
Returns the shape of an "n" dimentional array.
*/
function shape(m) {
    const result=[];
    while (Array.isArray(m)) {
        result.push(m.length);
        m=m[0];
    }
    return result;
}

In [9]:
testEq([0], shape([]));
testEq([1,0], shape([[]]));
testEq([1,1,3,2], shape([[[[0,1],[3,3],[4,4]]]]));

In [10]:
/**
Returns the mean of all elements in a 1d or 2d array.
*/
function mean(matrix) {
    if (!Array.isArray(matrix[0])) {
        matrix=[matrix];
    }
    const elementCount=shape(matrix).reduce((a,b)=>a*b);
    const sum=matrix.map(row=>row.reduce((a,b)=>a+b)).reduce((a,b)=>a+b);
    return sum/elementCount;
}

In [11]:
testEq(13/6, mean([0,1,2,3,3,4]));
testEq(13/6, mean([[0,1],[2,3],[3,4]]));

In [12]:
/**
Return a 1d or 2d array of `fillValue`.
*/
function full(d0,d1,fillValue) {
    if (d1 == null) {
        return new Array(d0).fill(fillValue);
    }
    const result=[];
    for (let i=0; i<d0; i++) {
        result.push(new Array(d1).fill(fillValue));
    }
    return result;
}

In [13]:
testEq([9],shape(full(9,null,3.3)));
testEq(9*3.3,Math.round(full(9,null,3.3).reduce((a,b)=>a+b)*10)/10);
testEq([9,2],shape(full(9,2))); // Don't do this unless you want a matrix of undefined (o:
testEq(3.4,full(3,2,3.4)[1][1]);

In [14]:
/**
Return a 1d or 2d array of zeros.
*/
function zeros(d0,d1) {
    return full(d0,d1,0);
}

In [15]:
testEq([9],shape(zeros(9)));
testEq(0,zeros(9).reduce((a,b)=>a-b));
testEq([9,2],shape(zeros(9,2)));
testEq(0,zeros(3,2)[1][1]);

In [16]:
/**
Return a square array with ones on the main diagonal.
*/
function identity(n) {
    const result=zeros(n,n);
    for (let i=0; i<n; i++) {
        result[i][i]=1;
    }
    return result;
}

In [17]:
testEq([[1,0,0], [0,1,0], [0,0,1]], identity(3));

In [18]:
/**
Return the mean and population standard deviation of a 1d array.

https://stackoverflow.com/questions/7343890/standard-deviation-javascript
*/
function meanAndStandardDeviation(a1d) {
    const n=a1d.length;
    const mean=a1d.reduce((a,b)=>a+b) / n;
    return [mean,Math.sqrt(a1d.map(a => Math.pow(a - mean, 2)).reduce((a, b) => a + b) / n)];
}

In [19]:
testEq([2, 1.414], round(meanAndStandardDeviation([0,1,2,3,4]), 3));

In [20]:
/**
Returns the transpose of a 2d array.
*/
function transpose(matrix) {
    const result = [];
    matrix.forEach(function(row,rowIndex) {
        row.forEach(function(elem,columnIndex) {
            if (rowIndex==0) {
                result[columnIndex]=[elem];
            } else {
                result[columnIndex].push(elem);
            }
        });
    });
    return result;
}

In [21]:
const left  = [
    [1,2,1],
    [0,1,0],
    [2,3,4]
];
const right = [
    [2,5],
    [6,7],
    [1,8]
];
testEq([[2,6,1],[5,7,8]],transpose(right));

In [22]:
/**
Returns a single value from a standard normal distribution.
*/
function randn_bm() {
    // Box-Muller transform - Max Collard - stack overflow
    var u=0, v=0;
    while(u==0) u=Math.random();
    while(v==0) v=Math.random();
    return Math.sqrt(-2.0*Math.log(u)) * Math.cos(2.0*Math.PI*v);
}

The following test checks that the mean of 100 `randn_bm` values is zero (after rounding to the nearest integer)

In [23]:
testEq(0, Math.round(Array(100).fill(0).map(_ => randn_bm()).reduce((a,b) => a+b)/100))

In [24]:
/**
Returns a 2d array filled with `randn_bm` values.
*/
function randn(d0,d1) {
    const result = [];
    for (let rowIndex = 0; rowIndex < d0; rowIndex++) {
        const row=[];
        result.push(row);
        for (let colIndex = 0; colIndex < d1; colIndex++) {
            row.push(randn_bm());
        }
    }
    return result;
}

In [25]:
let result=randn(20,5);
testEq([20,5],shape(result));
['Mean',result.map(row=>row.reduce((a,b)=>a+b)).reduce((a,b)=>a+b)/(20*5)];

[ 'Mean', 0.14603624249007172 ]


In [91]:
/**
Returns a 2d array filled with pseudo-random number in the range [low, high) with ~ uniform distribution.
*/
function uniform(d0,d1,low,high) {
    low = low || 0;
    high = high || 1;
    const result = [];
    for (let rowIndex = 0; rowIndex < d0; rowIndex++) {
        const row=[];
        result.push(row);
        for (let colIndex = 0; colIndex < d1; colIndex++) {
            row.push(Math.random()*(high-low)+low);
        }
    }
    return result;
}

In [26]:
/**
Return matrix of `newShape` if
- a is a scalar value,
- a is a 1d array with a length that matches newShape[1] or
- a is the new shape already.

`newShape` must be 2d.
*/
function reshape(a,newShape) {
    const oldShape=shape(a);
    if (oldShape.length==0) {
        return full(newShape[0],newShape[1],a);
    } else if (oldShape.length==1 && oldShape[0]==newShape[1]) {
        return new Array(newShape[0]).fill(a);
    }
    newShape.forEach((s,i) => {
        if (s!=oldShape[i]) throw new Error(`Can't reshape from [${oldShape}] to [${newShape}]`);
    });
    return a;
}

In [32]:
testEq([2,2],shape(reshape(1.23,[2,2])));
testEq([1,3],shape(reshape([1,2,3],[1,3])));
testEq([3],shape(reshape([1,2,3],[3]))); // 1d new shape works some if shape(a)==newShape
testEq([5,3],shape(reshape([1,2,3],[5,3])));
try {
    reshape([1,2,3],[5,2]);
    throw new Error('The reshape above should have failed');
} catch (e) {
    testEq("Can't reshape from [3] to [5,2]", e.message);
}

In [33]:
/**
Elementwise sum of a and b where a and b are 1d.
*/
function matrixSum1d(a,b) {
    return a.map((e,i) => e+b[i]);
}

/**
Elementwise sum of a2d and b, where a2d is 2d and b can be reshaped to match a.
*/
function matrixSum2d(a2d,b) {
    const b2d=reshape(b,shape(a2d));
    return a2d.map((row,i) => matrixSum1d(row, b2d[i]));
}

In [34]:
testEq([[1,3],[10,30]], matrixSum2d([[0,1],[9,28]],[1,2]));
testEq([[1,3],[12,32]], matrixSum2d([[0,1],[9,28]],[[1,2],[3,4]]));

In [35]:
/**
Element wise subtraction of `b` from `a`, where a and b are 1d.
*/
function matrixSubtract1d(a,b) {
    return a.map((e,i) => e-b[i]);
}

/**
Elementwise subtraction of b from a2d, where a2d is 2d and b can be reshaped to match a.
*/
function matrixSubtract2d(a2d,b) {
    const b2d=reshape(b,shape(a2d));
    return a2d.map((row,i) => matrixSubtract1d(row,b2d[i]));
}

In [36]:
testEq([0,-1,-2,-3,-4],matrixSubtract1d([1,1,1,1,1],[1,2,3,4,5]));
testEq([[0,-1,-2,-3,-4],[0,1,2,3,4]],matrixSubtract2d([[1,1,1,1,1],[1,3,5,7,9]],[1,2,3,4,5]));
testEq([[0,-1,-2,-3,-4],[0,1,2,3,4]],matrixSubtract2d([[1,1,1,1,1],[1,3,5,7,9]],[[1,2,3,4,5],[1,2,3,4,5]]));

In [37]:
/**
Element wise multiplication of `b` and `a`, where a is 1d and b can be reshaped to match a.
*/
function matrixMultiply1d(a1d,b) {
    const b1d=reshape(b,shape(a1d));
    return a1d.map((e,i) => e*b1d[i]);
}

/**
Elementwise multiplication of a2d with b, where a2d is 2d and b can be reshaped to match a.
*/
function matrixMultiply2d(a2d,b) {
    const b2d=reshape(b,shape(a2d));
    return a2d.map((row,i) => matrixMultiply1d(row,b2d[i]));
}

In [38]:
testEq([10,20,30],matrixMultiply1d([1,2,3],10));
testEq([10,4,3],matrixMultiply1d([1,2,3],[10,2,1]));
testEq([[10,20,30],[-10,-20,-30]],matrixMultiply2d([[1,2,3],[-1,-2,-3]],10));
testEq([[10,4,3],[-10,-4,-3]],matrixMultiply2d([[1,2,3],[-1,-2,-3]],[10,2,1]));
testEq([[10,4,3],[0,0,0]],matrixMultiply2d([[1,2,3],[-1,-2,-3]],[[10,2,1],[0,0,0]]));

In [39]:
/**
Returns the dot product of two 2d arrays.
See: http://matrixmultiplication.xyz/
*/
function dotProduct(a,b) {
    const bTransposed=transpose(b);
    return a.map((aRow,aRowIndex) => {
        return bTransposed.map((bRow) => {
            return matrixMultiply1d(aRow,bRow).reduce((a,b) => a+b);
        });         
    });
}

In [40]:
let actual=dotProduct(left,right)
testEq([shape(left)[0],shape(right)[1]], shape(actual));
testEq([[15,27],[6,7],[26,63]], actual);
testEq([[2,5],[0,0],[4,10]], dotProduct([[1],[0],[2]],[[2,5]]));

In [41]:
/**
Return the index of the highest value in `a`.
*/
function argmax(a) {
    return a.indexOf(Math.max(...a));
}

In [42]:
testEq(2, argmax([0,0,1]));
testEq(2, argmax([0,0,.5]));
testEq(1, argmax([0,0.51,.5]));

In [43]:
/**
Normalize a 2d array by subtracting its mean and dividing by its standard deviation for all elements.
*/
function normalize(a2d) {
    const [mean,std] = meanAndStandardDeviation(flatten(a2d));
    return matrixMultiply2d(matrixSubtract2d(a2d,mean), 1/std);
}

In [44]:
testEq([[1.41, -0.71, -0.71],
        [-0.71, 1.41, -0.71],
        [-0.71, -0.71, 1.41]], round(normalize(identity(3)), 2));
testEq([0,1], round(meanAndStandardDeviation(flatten(normalize(identity(3)))), 5));

In [ ]:
export {
    round,flatten,exp,shape,transpose,dotProduct,randn,uniform,full,zeros,mean,reshape,argmax,
    normalize,identity,meanAndStandardDeviation,
    matrixSum1d,matrixSum2d,matrixSubtract1d,matrixSubtract2d,matrixMultiply1d,matrixMultiply2d}